In [165]:
import sys
sys.path.append("../")
import models
import models_utils
from mcmc_utils import *
from utils import *

%matplotlib inline

In [166]:
import seaborn as sns
import math
import pandas as pd

In [167]:
m = models.PapaModelC8()
predict = m.predict
data = m.y
error = 0.1
weight_vector = m.domains

prior_min = [0.0001, 0.0001, 0.0001, 1., 3.5, 3.5, 3.5, 3.5]
prior_max = [1., 2., 2., 200., 9., 9., 9., 9.]

In [ ]:
param_values = []
SAMPLES = 5000
for index in range(SAMPLES):
    while(True):
        params = initialise(prior_min, prior_max)
        LL = calculate_log_likelihood(data, error, weight_vector, params, m.predict)
        if (not math.isnan(LL)):
            break
    params.append(LL)
    param_values.append(params)

In [ ]:
param_values = pd.DataFrame(param_values, columns = ["alpha", "D", "Co", "Ns", "K", "K1", "K2", "K3", "LL"])

In [ ]:
THRESH = -1500
print sum(param_values["LL"] > THRESH)
print param_values[param_values["LL"] > THRESH].mean()
print param_values[param_values["LL"] > THRESH].std()